# Homework Assignment #4 (Individual)
## Regression and SVM Predictive Models

### <p style="text-align: right;"> &#9989; Collin Dobson</p>
### <p style="text-align: right;"> &#9989; dobsonco</p>

### Goal for this homework assignment
By now, you have learned a bit about predictive models. In this assignment, you will practice:

* Building predictive models using `statsmodels` and `sklearn`
* Evaluating your predictive models

**This assignment is due roughly two weeks from now at 11:59 pm on Friday, April 21st.** It should be uploaded into the "Homework Assignments" submission folder for Homework #4.  Submission instructions can be found at the end of the notebook.


---
## Part 0: Add to your Git repository to track your progress on your assignment (4 points)

For this assignment, you're going to add it to the `cmse202-s23-turnin` repository you created in class so that you can track your progress on the assignment and preserve the final version that you turn in. In order to do this you need to

**&#9989; Do the following**:

1. Navigate to your `cmse202-s23-turnin` repository and create a new directory called `hw-04`.
2. Move this notebook into that **new directory** in your repository, then **add it and commit it to your repository**.
1. Finally, to test that everything is working, "git push" the file so that it ends up in your GitHub repository.

**Important**: Double check you've added your Professor and your TA as collaborators to your "turnin" repository (you should have done this in the previous homework assignment).

**Also important**: Make sure that the version of this notebook that you are working on is the same one that you just added to your repository! If you are working on a different copy of the notebook, **none of your changes will be tracked**!

If everything went as intended, the file should now show up on your GitHub account in the "`cmse202-s23-turnin`" repository inside the `hw-04` directory that you just created.  Periodically, **you'll be asked to commit your changes to the repository and push them to the remote GitHub location**. Of course, you can always commit your changes more often than that, if you wish.  It can be good to get into a habit of committing your changes any time you make a significant modification, or when you stop working on the project for a bit.

&#9989; **Do this**: Before you move on, put the command that your instructor should run to clone your repository in the markdown cell below.

git clone https://github.com/dobsonco/CMSE202-s23-turnin.git

# Building a Model to Predict if it Will Rain Tomorrow

In this dataset we have weather data for a number of Australian cities. We’re going to **build a model that uses data from today to predict if it will rain tomorrow.** To simplify our model, let’s just look at the weather for one city.

## Part 1. Working with the Data

The dataset that we’ll be using can be found here:
- `https://raw.githubusercontent.com/msu-cmse-courses/cmse202-S23-data/main/HW/HW4/aussie_weather_data.csv`

#### 1.1 (3 Points)
**Download the data set and read it in using Pandas.**

In [ ]:
!curl https://raw.githubusercontent.com/msu-cmse-courses/cmse202-S23-data/main/HW/HW4/aussie_weather_data.csv -o aussie_weather.csv
    

In [ ]:
import pandas as pd

In [ ]:
weather_data = pd.read_csv('aussie_weather.csv')
weather_data.head()

#### 1.2 (3 Points)
This dataset has weather information for multiple cities in Australia. It’s quite a large dataset. In fact, it’s a little *too* large for our purposes; it will take a considerable amount of time to train a model on so much data. So we’ll just work with the data for a specific city.

The first thing we’ll do is see which cities we have data for. 
**Print out a list of the specific cities in this dataset.** (Note that this is not the same as printing the entire `Location` column.)

In [ ]:
#Write your code here
cities = list(set(weather_data['Location']))
cities

#### 1.3 (3 Points)
Select one of the cities (and **please** don't just pick the first one!). Cut down your dataframe so that it only contains data from your chosen city. You should also **look through the columns in this dataset and ensure that all of the data is ready for our analysis/model creation.**

In [ ]:
#Write your code here
mask = weather_data['Location'] == 'WaggaWagga'
wagga_data = weather_data[mask]

wagga_data.iloc[:,4:]

#### 1.4 (3 Points)
Finally, before we start creating our models, let's first split our data into training and testing datasets. **Keep in mind what we want our model to predict and how this dataset gives you the information you need for your model.**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

In [ ]:
#Write your code here
train_data,test_data = train_test_split(wagga_data)

train_vectors = train_data.iloc[:,4:24]
train_labels = train_data['RainTomorrow']

test_vectors = test_data.iloc[:,4:24]
test_labels = test_data['RainTomorrow']

## 2.0 Building a Regression Model

#### 2.1 (3 Points)
We’ll start by creating a regression model. **Does it make sense for us to use linear regression or logistic regression? Explain your choice.**

It doesn't make sense to do a linear regression, but a logistic regression does make sense, as we want either a 1 or a 0. Logistic regressions are much better for binary decisions.

#### 2.2 (12 Points)
Create a regression model. Use the model summary to determine which parameters are most statistically significant in your model. If you get an error during at this stage it may be helpful to go back to part 1.3 and ensure that all of your data is appropriate for creating a model. 

In [118]:
#Write your code here
logit_model = sm.Logit(train_labels, sm.add_constant(train_vectors))
result = logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.232377
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:           RainTomorrow   No. Observations:                 1812
Model:                          Logit   Df Residuals:                     1791
Method:                           MLE   Df Model:                           20
Date:                Wed, 12 Apr 2023   Pseudo R-squ.:                  0.4931
Time:                        17:46:04   Log-Likelihood:                -421.07
converged:                       True   LL-Null:                       -830.74
Covariance Type:            nonrobust   LLR p-value:                1.102e-160
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            73.0859     19.086      3.829      0.000      35.678     110.494
MinTemp          -0.

In [119]:
predicted = result.predict(sm.add_constant(test_vectors))
predicted_int = np.rint(predicted)
predicted_int = predicted_int.astype('int64')

print(f'Accuracy: {metrics.accuracy_score(y_true=test_labels,y_pred=predicted_int)*100}%')

Accuracy: 90.06622516556291%


In [120]:
print(classification_report(test_labels, predicted_int))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94       488
           1       0.83      0.60      0.70       116

    accuracy                           0.90       604
   macro avg       0.87      0.79      0.82       604
weighted avg       0.90      0.90      0.89       604



*Record which features are most statistically significant.*

Mintemp, Evap, Wind Gust Dir, Wind Dir 3pm, Humidity 9 am, and Rain Today are the most important features.



#### 2.3 (8 Points)
**Fit your test data and create/print a confusion matrix. We’ll use this to evaluate how well your predicitve model performs.**

In [78]:
del train_vectors['MaxTemp']
del test_vectors['MaxTemp']

del train_vectors['Rainfall']
del test_vectors['Rainfall']

del train_vectors['Sunshine']
del test_vectors['Sunshine']

del train_vectors['WindGustSpeed']
del test_vectors['WindGustSpeed']

del train_vectors['WindDir9am']
del test_vectors['WindDir9am']

del train_vectors['WindSpeed9am']
del test_vectors['WindSpeed9am']

del train_vectors['Humidity3pm']
del test_vectors['Humidity3pm']

del train_vectors['Pressure9am']
del test_vectors['Pressure9am']

del train_vectors['Pressure3pm']
del test_vectors['Pressure3pm']

In [79]:
logit_model = sm.Logit(train_labels, sm.add_constant(train_vectors))
result = logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.313940
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:           RainTomorrow   No. Observations:                 1812
Model:                          Logit   Df Residuals:                     1800
Method:                           MLE   Df Model:                           11
Date:                Wed, 12 Apr 2023   Pseudo R-squ.:                  0.3385
Time:                        17:21:45   Log-Likelihood:                -568.86
converged:                       True   LL-Null:                       -859.96
Covariance Type:            nonrobust   LLR p-value:                8.967e-118
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -3.0081      0.958     -3.141      0.002      -4.885      -1.131
MinTemp          0.0438

In [80]:
#Write your code here
predicted = result.predict(sm.add_constant(test_vectors))
predicted_int = np.rint(predicted)
predicted_int = predicted_int.astype('int64')

print(f'Accuracy: {metrics.accuracy_score(y_true=test_labels,y_pred=predicted_int)*100}%')

Accuracy: 87.74834437086093%


In [81]:
print('Confusion Matrix:')
print(confusion_matrix(test_labels, predicted_int,labels=(0,1)))

Confusion Matrix:
[[488  19]
 [ 55  42]]


In [82]:
print(classification_report(test_labels, predicted_int))

              precision    recall  f1-score   support

           0       0.90      0.96      0.93       507
           1       0.69      0.43      0.53        97

    accuracy                           0.88       604
   macro avg       0.79      0.70      0.73       604
weighted avg       0.86      0.88      0.87       604



#### 2.4 (3 Points)
Look at/calculate the Precision and Recall of your model. **Which one is greater? Describe in plain language what that means about the performance of your model (I.e., the circumstances in which it does/doesn’t do well).**

The precision on average is higher than the recall. This means that the number of correct predictions was very high, while the number of positives returned for each varied. The recall score was higher for no rain because there was way more data for days that weren't rainy.

## 3.0 Interlude: A Base Rate Frequency Model (12 Points)

If we didn't create a fancy model for predicting future rainfall, would that mean that we had *no way* of predicting if it would rain? Of course not. There's always *some* kind of model we can use (they just won't be very sophisticated). 

In the case of our problem--predicting whether it will rain tomorrow--what is the simplest model we could use? We'd need to look at the **base rate frequency**. If we know that, over the course of the year, it rains ~30% of the time, that would be our base rate frequency. If we were to guess that it would be sunny every day of the year, we'd be wrong ~30% of the time, *but we'd be right ~70% of the time.* 

Why is this important? Imagine we create a sophisticated model that accurately predicts rain/no rain ~60% of the time; without any other information, we might think, "That's not too bad, I guess." It certainly feels better than *no* information, right? Now think about the base rate frequency model, which achieves an accuracy of **70%** just by guessing the same thing every time. Suddenly, our sophisticated model seems like crap! It tells us we could be doing ~10% better than our fancy model by guessing. This is why we use the base rate frequency as a simple way to test our models.


**Calculate how frequently it rains in your city. If you were to guess that it *wasn't* going to rain every day, how frequently would you be right? (We'll use this as our baseline model.)**


In [83]:
#Write your code for calculating the frequency of rain here.
print(f'Total Days: {len(wagga_data["RainToday"])}')

print(f'Total Rainy Days: {len(np.where(wagga_data["RainToday"] == 1)[0])}')

print(f'Rainy Day Percent = {len(np.where(wagga_data["RainToday"] == 1)[0]) / len(wagga_data["RainToday"]) * 100}%')

print(f'There should be {len(np.where(wagga_data["RainToday"] == 1)[0]) / len(wagga_data["RainToday"]) * 365} rainy days a year')

Total Days: 2416
Total Rainy Days: 419
Rainy Day Percent = 17.34271523178808%
There should be 63.3009105960265 rainy days a year


We would only be right 17% of the time, so not great. This is significantly lower than the logistic model I made above.

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

## 4.0 SVM Model

We’ve seen how regression faired in predicting rain; now let’s give Support Vector Machines a shot. 

It’s not clear, *a priori*, which type of kernel (linear or RBF) would fair better. Therefore, we’ll need to test out both kernel types, as well as multiple values for hyperparameters ($C$ and $\gamma$). 

#### 4.1 (12 Points)
Run a grid search over both types of kernels and multiple values of $C$ and $\gamma$ (we recommend [0.01,0.1,1.0,10.0] for both). We *STRONGLY* recommend you set the parameter `n_jobs=-1` in `GridSearchCV`, which will distribute the computational load. Make sure to write down your best-fit hyperparameters. 

In [113]:
weather_data = pd.read_csv('aussie_weather.csv')

mask = weather_data['Location'] == 'WaggaWagga'
wagga_data = weather_data[mask]

train_data,test_data = train_test_split(wagga_data,random_state=3)

train_vectors = train_data.iloc[:,4:24]
train_labels = train_data['RainTomorrow']

test_vectors = test_data.iloc[:,4:24]
test_labels = test_data['RainTomorrow']

In [114]:
#Write your code here
param_grid = {'C': [0.01,0.1,1.0,10.0],
              'gamma': [0.01,0.1,1.0,10.0],
              'kernel': ['linear']}#,'rbf']}

clf = GridSearchCV(SVC(class_weight='balanced'), param_grid, n_jobs=-1)

clf = clf.fit(train_vectors, train_labels)

{'C': 0.1, 'gamma': 0.01, 'kernel': 'linear'}

#### 4.2 (8 Points)
Using your best-fit parameters, classify your test data and print off the confusion matrix. Note/calculate the Precision and Recall for your SVM model, as well as the overall accuracy.

$\mathrm{Accuracy} = \frac{\mathrm{Accurate~Predictions}}{\mathrm{Total~Samples}}$.

In [115]:
pred_labels = clf.predict(test_vectors)

In [116]:
print(classification_report(test_labels, pred_labels))

              precision    recall  f1-score   support

           0       0.97      0.87      0.92       488
           1       0.62      0.91      0.73       116

    accuracy                           0.87       604
   macro avg       0.80      0.89      0.83       604
weighted avg       0.91      0.87      0.88       604



In [117]:
zipped = zip(test_labels,pred_labels)

correct = 0
total = 0
for i,j in zipped:
    total += 1
    if i == j:
        correct += 1
        
print(f'Accuracy: {correct/total * 100}%')

Accuracy: 87.41721854304636%


## 5. Evaluation of Models

Here we’re going to investigate how well our predictive models actually work.


#### 5.1 (5 Points)

Compare the accuracy of both of your predictive models (regression and SVM) to the baseline model from part 3. Do your predictive models do better than the baseline model? Is their performance relative to the baseline model what *you* would expect? 


Both of these models are significantly more accurate than thr baseline model. Significantly so, in fact they perform better than I would have expected.

#### 5.2 (5 Points)

Let’s test the robustness of your model. Before you do anything, you should record the precision, recall, and accuracy of your two predictive models (if you haven’t already). Once you’ve done that, go all the way back to part 1.4 and change the random seed for `train_test_split`. Using this new data split, rerun all of your code from parts 2 and 4. For each new random seed, record the precision, recall, and accuracy for both models (and write them, as well as the random seed, in the cell below!). Do this for 3-5 different random seeds. 


* Random_state = 1
Logistic: A = 92, P = (0.94,0.79), R = (0.97,0.65)
SVM: A = 87, P = (0.97,0.54), R = (0.87,0.86)

* Random_state = 2
Logistic: A = 88, P = (0.90,0.69), R = (0.96,0.43)
SVM: A = 86, P = (0.97,0.58), R = (0.86,0.90)

* Random_state = 3
Logistic: A = 92, P = (0.94,0.79), R = (0.96,0.43)
SVM: A = 88, P = (0.90,0.69), R = (0.87,0.91)

#### 5.3 (5 Points)

How much variation did you observe in the precision, recall, and accuracy in your models? After going through this exercise, how (if at all) does this change your response to part 5.1? 

There was some variation, but not too much. It was a maximum of a few percent for the logistic and about one point for the SVM. They still perform much better than the baseline model, and at this point my expectations have risen.

#### 5.4 (5 Points)

Finally, compare the performance of your regression and SVM models. Which, if any, performed better? Esxplain.

The logistic regression edged out the SVM by a few percentage points in all situations. If I was going to pick a model to roll with, it would be the logistic regression.

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

### Congratulations, you're done!

&#169; Copyright 2022,  Department of Computational Mathematics, Science and Engineering at Michigan State University